# Benchmark avec River

## Consigne

* Accédez à River, rubrique API Benchmark, et prenez connaissance des méthodes
évaluées.
* Pré-sélectionnez les méthodes disponibles :
* * Régression : toutes les méthodes.
* * Classification multi-classes : sélectionnez 3 méthodes et 3 jeux de données.
* * Classification binaire : sélectionnez les méthodes dont le temps d’exécution < 1000 secondes.
* Exécutez les 3 types de méthodes sur River pour vérifier la compilation et reportez les
résultats du benchmark existant.
* Présentez une table indiquant en gras les meilleures performances et ajoutez une analyse comparative par dataset.


## Approche

Pour effectuer ce TP, étant donné le nombres importants d'algorithmes et de datasets, on va faire du multi-threading pour exécuter les benchmarks en parallèle.

On va s'inspirer de ce qui est fait dans River pour exécuter les benchmarks.

In [ ]:
from __future__ import annotations

import sys

import copy
import itertools
import json
import logging
import multiprocessing

import pandas as pd
from config import MODELS, N_CHECKPOINTS, TRACKS
from tqdm import tqdm

from river import metrics
from river.evaluate import Track


## Manage logging : make sure only warnings and errors are logged
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

MODELS["Binary classification"].update(MODELS["Multiclass classification"])
details = {}

In [14]:
def run_dataset(model_str, no_dataset, track, n_checkpoints=N_CHECKPOINTS):
    """ Run a model on a dataset and return the results.

    Args:
        model_str (_type_): model name
        no_dataset (_type_): dataset index
        track (Track) : binary, multiclass or regression in our case

    Returns:
        results :
            step : int
            track : str
            model : str
            dataset : str
            ... metric values ...
    """
    if isinstance(track, int):
        track = TRACKS[track]
    
    if isinstance(model_str, str):    
        model_name = model_str    
        model = MODELS[track.name][model_name].clone()
    else:
        model_name = model_str.__class__.__name__
        model = model_str.clone()
    
    dataset = track.datasets[no_dataset]
    print(f"Processing {model_str} on {dataset.__class__.__name__}")

    results = []
    track = copy.deepcopy(track)
    time = 0.0
    
    # Run the model on the dataset with a progress bar and N_CHECKPOINTS checkpoints
    for i in tqdm(
        track.run(model, dataset, n_checkpoints=n_checkpoints),
        total=n_checkpoints,
        desc=f"{model_str} on {dataset.__class__.__name__}",
    ):
        time += i["Time"].total_seconds()
        res = {
            "step": i["Step"],
            "track": track.name,
            "model": model_name,
            "dataset": dataset.__class__.__name__,
        }
        for k, v in i.items():
            if isinstance(v, metrics.base.Metric):
                res[k] = v.get()
        res["Memory in Mb"] = i["Memory"] / 1024**2
        res["Time in s"] = time
        results.append(res)
    return results

In [ ]:


def run_track(models: list[str], track: Track, n_workers: int = 50, n_checkpoints: int = N_CHECKPOINTS, pickle=True):
    """ Run a track with multiple models in parallel.

    Args:
        models (list[str]): list of model names
        track (Track): track 
        n_workers (int, optional): number of parallel workers. Defaults to 50.
    """
    if isinstance(track, int):
        track = TRACKS[track]
        


    pool = multiprocessing.Pool(processes=n_workers)
    runs = list(itertools.product(models, range(len(track.datasets)), [track], [n_checkpoints]))
    results = []

    for val in pool.starmap(run_dataset, runs):
        results.extend(val)

    
    csv_name = track.name.replace(" ", "_").lower()
    pd.DataFrame(results).to_csv(f"./{csv_name}.csv", index=False)


## Regression

In [ ]:
from river.evaluate import RegressionTrack

track = RegressionTrack()

details[track.name] = {"Dataset": {}, "Model": {}}
for dataset in track.datasets:
    details[track.name]["Dataset"][dataset.__class__.__name__] = repr(dataset)
for model_name, model in MODELS[track.name].items():
    details[track.name]["Model"][model_name] = repr(model)
with open("details.json", "w") as f:
    json.dump(details, f, indent=2)
run_track(models=MODELS[track.name].keys(), track=track, n_workers=50)


Processing Passive-Aggressive Regressor, mode 1 on ChickWeightsProcessing Stochastic Gradient Tree on TrumpApprovalProcessing Linear Regression on ChickWeightsProcessing Linear Regression on TrumpApprovalProcessing Aggregated Mondrian Forest on ChickWeightsProcessing Passive-Aggressive Regressor, mode 1 on TrumpApproval
Processing Aggregated Mondrian Forest on TrumpApproval
Processing Passive-Aggressive Regressor, mode 2 on TrumpApprovalProcessing k-Nearest Neighbors on TrumpApproval
Processing Passive-Aggressive Regressor, mode 2 on ChickWeights
Processing Linear Regression with l2 regularization on TrumpApproval
Processing Linear Regression with l1 regularization on TrumpApprovalProcessing Hoeffding Tree on TrumpApprovalProcessing Linear Regression with l2 regularization on ChickWeights
Processing Linear Regression with l1 regularization on ChickWeightsProcessing Hoeffding Tree on ChickWeightsProcessing k-Nearest Neighbors on ChickWeights


Linear Regression on TrumpApproval:   0%|          | 0/50 [00:00<?, ?it/s]





Processing Hoeffding Adaptive Tree on ChickWeights



Aggregated Mondrian Forest on ChickWeights:   0%|          | 0/50 [00:00<?, ?it/s]

Passive-Aggressive Regressor, mode 2 on ChickWeights:   0%|          | 0/50 [00:00<?, ?it/s]

...

Processing Bagging on TrumpApproval


Exponentially Weighted Average on ChickWeights:   0%|          | 0/50 [00:00<?, ?it/s]

Bagging on TrumpApproval:   0%|          | 0/50 [00:00<?, ?it/s]] [00:00<?, ?it/s]

Processing Bagging on ChickWeights


[baseline] Mean predictor on ChickWeights: 53it [00:00, 1183.24it/s]             
Linear Regression on TrumpApproval: 51it [00:00, 556.29it/s]              
Passive-Aggressive Regressor, mode 1 on ChickWeights: 53it [00:00, 656.19it/s]              
Linear Regression on ChickWeights: 53it [00:00, 618.50it/s]              
Linear Regression with l2 regularization on TrumpApproval:  92%|█████████▏| 46/50 [00:00<00:00, 452.69it/s]
Streaming Random Patches on TrumpApproval:   2%|▏         | 1/50 [00:00<00:08,  5.86it/s]
Adaptive Model Rules on ChickWeights: 53it [00:00, 271.72it/s]                        00<00:00, 211.45it/s]
Stochastic Gradient Tree on TrumpApproval:  24%|██▍       | 12/50 [00:00<00:00, 39.53it/s]             
Hoeffding Tree on TrumpApproval: 51it [00:00, 149.80it/s]                        7.34it/s]]
Hoeffding Adaptive Tree on ChickWeights: 53it [00:00, 130.72it/s]0:00<00:02, 18.21it/s].29it/s]
Hoeffding Adaptive Tree on TrumpApproval: 51it [00:00, 115.24it/s]            

## Binary Classification


In [11]:
from river.evaluate import BinaryClassificationTrack

track = BinaryClassificationTrack()

details[track.name] = {"Dataset": {}, "Model": {}}
for dataset in track.datasets:
    details[track.name]["Dataset"][dataset.__class__.__name__] = repr(dataset)
for model_name, model in MODELS[track.name].items():
    details[track.name]["Model"][model_name] = repr(model)
with open("details.json", "w") as f:
    json.dump(details, f, indent=2)
run_track(models=MODELS[track.name].keys(), track=track, n_workers=50)


Processing Logistic regression on BananasProcessing Aggregated Mondrian Forest on PhishingProcessing Aggregated Mondrian Forest on Elec2Processing Aggregated Mondrian Forest on BananasProcessing Aggregated Mondrian Forest on SMTPProcessing Logistic regression on Elec2
Processing ALMA on SMTPProcessing Logistic regression on SMTPProcessing sklearn SGDClassifier on Phishing
Processing ALMA on Elec2
Processing sklearn SGDClassifier on Elec2Processing Logistic regression on Phishing


Processing Naive Bayes on Elec2Processing Naive Bayes on Bananas
Processing sklearn SGDClassifier on Bananas
Processing ALMA on Phishing
Processing ALMA on BananasProcessing Naive Bayes on SMTP



Processing Naive Bayes on Phishing


Logistic regression on Bananas:   0%|          | 0/50 [00:00<?, ?it/s]

Logistic regression on Elec2:   0%|          | 0/50 [00:00<?, ?it/s]


Processing Hoeffding Tree on Bananas

Aggregated Mondrian Forest on Phishing:   0%|          | 0/50 [00:00<?, ?it/s]

...

Naive Bayes on SMTP:  24%|██▍       | 12/50 [00:11<00:25,  1.47it/s] 6.67it/s]

sklearn SGDClassifier on SMTP:   8%|▊         | 4/50 [00:11<02:08,  2.80s/it]              


Processing [baseline] Last Class on SMTP

Hoeffding Adaptive Tree on SMTP:  44%|████▍     | 22/50 [00:11<00:16,  1.68it/s]

Hoeffding Tree on SMTP:  70%|███████   | 35/50 [00:11<00:05,  2.80it/s]
finished run
number of examples = 90624
weighted example sum = 90624.000000
weighted label sum = -6838.000000
average loss = 0.585391
best constant = -0.150909
best constant's loss = 0.977226
total feature number = 813498

finished run
number of examples = 0
weighted example sum = 0.000000
weighted label sum = 0.000000
average loss = n.a.
total feature number = 0
k-Nearest Neighbors on Phishing:  72%|███████▏  | 36/50 [00:17<00:04,  3.24it/s]
Logistic regression on SMTP: 51it [00:18,  2.80it/s]                        s]]2it/s]]t/s]
Vowpal Wabbit logistic regression on SMTP: 51it [00:18,  2.80it/s]                        
Streaming Random Patches on Phishing:  84%|████████▍ | 42/50 [00:18<00:04,  1.66it/s]
finished run
number of examples = 190312
weighted example sum = 190312.000000
weighted label sum = -95096.000000
average loss = 0.000643
best constant = -0.999369
best constant's loss = 0.001261
total feature numb

## Multiclass Classification

In [ ]:
from river import (ensemble, forest, preprocessing, neighbors)

from river.evaluate import MultiClassClassificationTrack

track = MultiClassClassificationTrack()
track_no = 1
models = {"Adaptive Random Forest": forest.ARFClassifier(seed=42),
        "k-Nearest Neighbors": preprocessing.StandardScaler() | neighbors.KNNClassifier(),

        "Streaming Random Patches": ensemble.SRPClassifier()}

details[track.name] = {"Dataset": {}, "Model": {}}
for dataset in track.datasets:
    details[track.name]["Dataset"][dataset.__class__.__name__] = repr(dataset)
for model_name, model in MODELS[track.name].items():
    details[track.name]["Model"][model_name] = repr(model)
with open("details.json", "w") as f:
    json.dump(details, f, indent=2)
run_track(models=models.keys(), track=track_no, n_workers=50, pickle=True)


Processing Streaming Random Patches on ImageSegmentsProcessing Adaptive Random Forest on KeystrokeProcessing Adaptive Random Forest on Insects
Processing k-Nearest Neighbors on KeystrokeProcessing Streaming Random Patches on KeystrokeProcessing k-Nearest Neighbors on Insects

Processing Streaming Random Patches on InsectsProcessing Adaptive Random Forest on ImageSegments


Processing k-Nearest Neighbors on ImageSegments



Streaming Random Patches on ImageSegments:   0%|          | 0/50 [00:00<?, ?it/s]

Adaptive Random Forest on Insects:   0%|          | 0/50 [00:00<?, ?it/s]s]it/s]67it/s]s]


Adaptive Random Forest on ImageSegments: 51it [00:03, 13.79it/s]                        ]
k-Nearest Neighbors on ImageSegments: 51it [00:06,  7.43it/s]                        it/s]
Adaptive Random Forest on Keystroke: 100%|██████████| 50/50 [00:15<00:00,  3.32it/s]4it/s]
Streaming Random Patches on ImageSegments: 51it [00:20,  2.46it/s]                        
k-Nearest Neighbors on Keystroke:  60%|██████    | 30/50 [00:45<00:29,  1.47s/it]s/it]Process ForkPoolWorker-396:
Process ForkPoolWorker-397:
Process ForkPoolWorker-394:
Process ForkPoolWorker-390:
Streaming Random Patches on Keystroke:  34%|███▍      | 17/50 [00:45<01:28,  2.67s/it]Process ForkPoolWorker-375:
Process ForkPoolWorker-395:
Process ForkPoolWorker-379:
Process ForkPoolWorker-357:
Process ForkPoolWorker-380:
Process ForkPoolWorker-352:
Process ForkPoolWorker-385:
Process ForkPoolWorker-383:
Process ForkPoolWorker-398:
Process F

KeyboardInterrupt: 

Process ForkPoolWorker-355:
Process ForkPoolWorker-365:
Process ForkPoolWorker-368:
Process ForkPoolWorker-386:
Process ForkPoolWorker-370:
Process ForkPoolWorker-376:
Process ForkPoolWorker-360:
Process ForkPoolWorker-377:
Process ForkPoolWorker-382:
Process ForkPoolWorker-366:


Process ForkPoolWorker-351:
Process ForkPoolWorker-358:
Process ForkPoolWorker-392:
Process ForkPoolWorker-372:

k-Nearest Neighbors on Keystroke:  60%|██████    | 30/50 [00:45<00:30,  1.51s/it]Process ForkPoolWorker-361:
Process ForkPoolWorker-367:
Process ForkPoolWorker-391:
Process ForkPoolWorker-381:
Process ForkPoolWorker-369:
Process ForkPoolWorker-364:
Process ForkPoolWorker-363:
Process ForkPoolWorker-373:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-362:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call l

dict_keys(['Adaptive Random Forest', 'AdaBoost', 'Streaming Random Patches'])
